# Add a short AI summary to each vote

In [ ]:
import openai
from dotenv import load_dotenv
import os
import json
from openai import OpenAI
from datetime import datetime

load_dotenv()

In [25]:
def _format_vote_all(vote: dict[str, str]) -> str:
    return f"""
    # Abstimmung im Deutschen Bundestag

    ### {vote["title"]}

    {vote["subtitle"]}

    ### Details

    {vote["detail_text"]}
    """


def _format_vote_title(vote: dict[str, str]) -> str:
    return f"""
    # {vote["title"]}

    {vote["subtitle"]}
    """


def _format_party_vote(
    titel: str, subtitle: str, detail_text: str, party: str, voting: str
) -> str:
    return f"""
    # {party} hat '{voting}' abgestimmt

    ## {titel}

    {subtitle}

    {detail_text}
    
    Deine extrahierte Begründung der Partei {party} für ihre Entscheidung:
    """


prompt_short_description = """
    # Rolle

    Du bist ein Experte, der aus der detaillierten Dokumentation einer Abstimmung im Deutschen Bundestag nur den reinen Inhalt des Gesetzesvorschlags oder Themas der Abstimmung ermittelt, ohne die Debattenbeiträge, Meinungen oder Positionen der Beteiligten im Detail darzustellen.

    # Deine Handlungsanweisungen

    Du erhältst Titel, Untertitel und die detaillierte Beschreibung einer Bundestagsdebatte. Extrahiere ausschließlich folgende Punkte:
    - Worum geht es im Kern (Thema oder Gegenstand des Gesetzes/Antrags/Forderung)?
    - Welche konkreten Regelungen, Inhalte und Ziele sind in dem Gesetz/Antrag/Forderung enthalten?
    - Welche Bedingungen/Voraussetzungen müssen erfüllt sein (falls erwähnt)?
    - Welche Konsequenzen oder Auswirkungen folgen aus dem Gesetz/Antrag/Forderung (falls erwähnt)?
    - Ignoriere komplette was in der Debatte gesagt oder von einzelnen Personen als Meinung wiedergegeben wurde oder wie allgemein abgestimmt wurde.
    - Gib die extrahierten Informationen in einer prägnanten Zusammenfassung wieder, die nur den Kerninhalt des Gesetzes/Antrags/Forderung beschreibt.
    - Nutze keine Überschriften, fette Text, Aufzählungen, Stichpunkte oder sonstige Formatierung. Schreibe im reinen Fließtext nur mit Absätzen.
    """


prompt_submitting_party = """
    # Rolle

    Du bist ein Experte, der aus der detaillierten Dokumentation einer Abstimmung im Deutschen Bundestag nur den Partei ermittelt, die den Gesetzesvorschlag eingereicht hat. 

    # Deine Handlungsanweisungen

    Du erhältst Titel, Untertitel und die detaillierte Beschreibung einer Bundestagsdebatte. Extrahiere ausschließlich folgende Punkte:
    - Welche Partei hat den Gesetzesvorschlag eingereicht?
    - Antworte nur mit der Partei, keine weiteren Informationen. Antworte mit der Abkürzung der Partei, zum Beispiel "SPD" oder "CDU".
    - Wenn mehrere Parteien den Gesetzesvorschlag eingereicht haben, antworte mit allen Parteien, getrennt durch Kommas.
    - Lass dich nicht verwirren von Formulierungen wie Antrag der CDU zum Gesetzentwurf der SPD. In solchen Fällen geht es immer un die Partei welchen den jüngsten Antrag eingereicht hat, also hier die CDU (Da es erst den Gesetzentwurf der SPD und dann den Antrag der CDU gibt).
    """

prompt_vote_category = """
    # Rolle

    Du bist ein Experte, der aus der detaillierten Dokumentation einer Abstimmung im Deutschen Bundestag nur die Kategorie ermittelt, zu der die Abstimmung gehört. Wir unterscheiden zwischen folgenden Kategorien:
    - G: Gesetzentwürfe
    - SA: Selbständige Anträge
    - EA: Entschließungsanträge
    - AA: Änderungsanträge
    - B: Beschlussempfehlungen

    # Antragsarten

    ### Gesetzentwürfe
    Gesetzentwürfe sind die Grundlage für neue Gesetze oder Gesetzesänderungen. Sie durchlaufen in der Regel drei Lesungen im Bundestag, bevor darüber abgestimmt wird.

    ### Selbständige Anträge
    Können von Fraktionen oder mindestens 5 Prozent der Bundestagsmitglieder eingebracht werden und dienen dazu, den Bundestag zu einer bestimmten Entscheidung aufzufordern. Können sich auf verschiedene politische Themen beziehen.

    ### Entschließungsanträge
    Werden häufig in der dritten Lesung von Gesetzentwürfen eingebracht und können auch zu Regierungserklärungen, Regierungsberichten oder Großen Anfragen vorgelegt werden. Dienen oft dazu, eine politische Position zu einem Thema zu verdeutlichen.

    ### Änderungsanträge
    Zielen darauf ab, Änderungen an bestehenden Gesetzentwürfen oder Anträgen vorzunehmen.

    ### Beschlussempfehlungen
    Beschlussempfehlungen werden von Ausschüssen vorgelegt, nachdem diese einen Auftrag vom Plenum erhalten haben, sich mit einem bestimmten Thema zu befassen

    # Deine Handlungsanweisungen

    Du erhältst Titel, Untertitel und die detaillierte Beschreibung einer Bundestagsdebatte. Extrahiere ausschließlich folgende Punkte:
    - Welche Kategorie gehört zu der Abstimmung?
    - Antworte nur mit der Kategorie, keine weiteren Informationen. Antworte mit der Abkürzung der Kategorie, zum Beispiel "G", "SA", "EA", "AA" oder "B".
    """


def _format_prompt_vote_justifications(party: str) -> str:
    return f"""
# Rolle

Du bist ein Experte, der aus der detaillierten Dokumentation einer Abstimmung im Deutschen Bundestag die Begründung der Partei {party} für ihr Abstimmverhalten ermittelt.

# Deine Handlungsanweisungen

Du erhältst die folgenden Informationen:
- Abstimmungsergebnisse der Partei
- Titel und Untertitel der Abstimmung
- Detaillierte Mitschriften der Debatte

Extrahiere ausschließlich folgende Punkte:
- Welche Begründung hat die Partei für ihre Entscheidung gegeben oder wie kann man ihre Entscheidung erklären aufgrund der Aussagen der Abgeordneten?
- Wenn du keine Begründung finden kannst, antworte mit "Keine Begründung gefunden".
- Antworte nur mit der Begründung welche die Partei für ihre Entscheidung gegeben hat, keine weiteren Informationen.
- Füge keinen Allgemeinen Informationen über die Debatte oder die Abstimmungsergebnisse von anderen Parteien hinzu.
- Wenn die Informationen nicht ausreichen füge keine Interpretationen hinzu welche sich nicht klar aus dem Text ergibt! Schreibe dann "Keine Begründung gefunden".
"""

In [26]:
client = OpenAI()


def query_openai(vote: str, prompt: str) -> str:
    response = client.chat.completions.create(
        model="chatgpt-4o-latest",
        messages=[
            {"role": "user", "content": prompt + "\n\n" + vote},
        ],
    )
    return response.choices[0].message.content

In [27]:
def add_short_description(vote_id: int) -> None:
    with open(f"../votes/vote_{vote_id}.json", "r", encoding="utf-8") as f:
        vote = json.load(f)

    if (
        vote is None
        or vote["detail_text"] is None
        or vote["title"] is None
        or vote["date"] is None
    ):
        print(f"Skipping vote {vote_id} because it is missing information")
        return

    short_description = query_openai(_format_vote_all(vote), prompt_short_description)
    vote["short_description"] = short_description
    with open(f"../votes/vote_{vote_id}.json", "w", encoding="utf-8") as f:
        json.dump(vote, f, indent=4, ensure_ascii=False)

In [28]:
def parse_german_date(date_str: str) -> datetime:
    """Convert German date format to datetime object"""
    german_months = {
        "Januar": "01",
        "Februar": "02",
        "März": "03",
        "April": "04",
        "Mai": "05",
        "Juni": "06",
        "Juli": "07",
        "August": "08",
        "September": "09",
        "Oktober": "10",
        "November": "11",
        "Dezember": "12",
    }

    # Clean up the input string
    date_parts = date_str.strip().split()
    day = date_parts[0].replace(".", "")
    month = german_months[date_parts[1]]
    year = date_parts[2]

    # Create standardized date string
    standard_date = f"{year}-{month}-{day.zfill(2)}"
    return datetime.strptime(standard_date, "%Y-%m-%d")

In [29]:
def add_submitting_party(vote_id: int) -> None:
    with open(f"../votes/vote_{vote_id}.json", "r", encoding="utf-8") as f:
        vote = json.load(f)

    if (
        vote is None
        or vote["detail_text"] is None
        or vote["title"] is None
        or vote["date"] is None
    ):
        print(f"Skipping vote {vote_id} because it is missing information")
        return

    submitting_party = query_openai(_format_vote_title(vote), prompt_submitting_party)

    if submitting_party is None or submitting_party == "":
        print(
            f"Skipping vote {vote_id} because the submitting party is not clear: {submitting_party}"
        )
        return

    print(submitting_party)
    submitting_parties = submitting_party.split(",")
    clean_submitting_parties = []
    for party in submitting_parties:
        party = party.strip().upper()
        if "CDU" in party or "CDU/CSU" in party or "CHRISTLICH" in party:
            clean_submitting_parties.append("CDU")
        elif "SPD" in party or "SOZIALDEMOKRATISCHE" in party:
            clean_submitting_parties.append("SPD")
        elif "FDP" in party or "FREIE DEMOKRATEN" in party:
            clean_submitting_parties.append("FDP")
        elif "GRÜNE" in party or "BÜNDNIS" in party:
            clean_submitting_parties.append("GRÜNE")
        elif "LINKE" in party or "DIE LINKE" in party:
            clean_submitting_parties.append("LINKE")
        elif "AFD" in party or "ALTERNATIVE" in party:
            clean_submitting_parties.append("AFD")
        elif "REGIERUNG" in party or "BUNDESREGIERUNG" in party or "BUNDES" in party:
            if parse_german_date(vote["date"]) > datetime(2021, 12, 7):
                clean_submitting_parties.append("SPD")
                clean_submitting_parties.append("FDP")
                clean_submitting_parties.append("GRÜNE")
            else:
                clean_submitting_parties.append("CDU")
                clean_submitting_parties.append("SPD")

    vote["submitting_parties"] = clean_submitting_parties

    if "submitting_party" in vote:
        del vote["submitting_party"]

    with open(f"../votes/vote_{vote_id}.json", "w", encoding="utf-8") as f:
        json.dump(vote, f, indent=4, ensure_ascii=False)

In [30]:
def add_vote_category(vote_id: int) -> None:
    with open(f"../votes/vote_{vote_id}.json", "r", encoding="utf-8") as f:
        vote = json.load(f)

    vote_category = query_openai(_format_vote_title(vote), prompt_vote_category)

    if vote_category is None or vote_category == "" or len(vote_category) > 2:
        print(
            f"Skipping vote {vote_id} because the vote category is not clear: {vote_category}"
        )
        return

    vote["vote_category"] = vote_category

    with open(f"../votes/vote_{vote_id}.json", "w", encoding="utf-8") as f:
        json.dump(vote, f, indent=4, ensure_ascii=False)

In [31]:
from data.scripts.script_utils import convert_party_short_hand_to_party_id


def add_vote_justifications(vote_id: int) -> None:
    with open(f"../votes/vote_{vote_id}.json", "r", encoding="utf-8") as f:
        vote = json.load(f)

    if (
        vote is None
        or vote["detail_text"] is None or vote["detail_text"] == ""
        or vote["title"] is None
        or vote["date"] is None
    ):
        print(f"Skipping vote {vote_id} because it is missing information")
        return

    if vote["vote_category"] == "B":
        print(f"Skipping vote {vote_id} because it is a Beschlussempfehlung")
        return

    voting_results = vote["voting_results"]
    result_by_party = voting_results["by_party"]
    parties = [result["party"] for result in result_by_party]

    vote_justifications = {}

    for party in parties:
        if party == "FRAKTIONSLOS":
            continue
        
        party_vote = [result for result in result_by_party if result["party"] == party][0]
        party_yes = party_vote["yes"]
        party_no = party_vote["no"]
        party_abstain = party_vote["abstain"]

        party_decision = max(int(party_yes), int(party_no), int(party_abstain))
        if party_decision == int(party_yes):
            party_justification = "Dafür"
        elif party_decision == int(party_no):
            party_justification = "Dagegen"
        else:
            party_justification = "Enthaltung"

        party_vote_justifications = query_openai(
            _format_party_vote(
                vote["title"],
                vote["subtitle"],
                vote["detail_text"],
                party,
                party_justification,
            ),
            _format_prompt_vote_justifications(party),
        )

        if party_vote_justifications is None or party_vote_justifications == "":
            print(
                f"Skipping vote {vote_id} because the party vote justification is not clear: {party_vote_justifications}"
            )
            return

        vote_justifications[convert_party_short_hand_to_party_id(party)] = (
            party_vote_justifications
        )

    vote["vote_justifications"] = vote_justifications

    with open(f"../votes/vote_{vote_id}.json", "w", encoding="utf-8") as f:
        json.dump(vote, f, indent=4, ensure_ascii=False)

In [32]:
def fix_naming_in_vote(vote_id: int) -> None:
    with open(f"../votes/vote_{vote_id}.json", "r", encoding="utf-8") as f:
        vote = json.load(f)

    for voting_result in vote["voting_results"]["by_party"]:
        party = voting_result["party"].strip().upper()
        if "CDU" in party or "CDU/CSU" in party or "CHRISTLICH" in party:
            voting_result["party"] = "CDU"
        elif "SPD" in party or "SOZIALDEMOKRATISCHE" in party:
            voting_result["party"] = "SPD"
        elif "FDP" in party or "FREIE DEMOKRATEN" in party:
            voting_result["party"] = "FDP"
        elif "GRÜNE" in party or "BÜNDNIS" in party:
            voting_result["party"] = "GRÜNE"
        elif "LINKE" in party or "DIE LINKE" in party:
            voting_result["party"] = "LINKE"
        elif "AFD" in party or "ALTERNATIVE" in party:
            voting_result["party"] = "AFD"
        elif "BSW" in party:
            voting_result["party"] = "BSW"
        elif "FRAKTIONSLOS" in party:
            voting_result["party"] = "FRAKTIONSLOS"

    with open(f"../votes/vote_{vote_id}.json", "w", encoding="utf-8") as f:
        json.dump(vote, f, indent=4, ensure_ascii=False)

In [ ]:
for vote_id in range(377, 940):
    try:
        add_vote_justifications(vote_id)
    except Exception as e:
        print(f"Error fixing naming in vote {vote_id}: {e}")